[Node 15: SWIG](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node15.html)

Navigation:

**Next:** [Threads and Multi–Processing](node16.ipynb) **Up:** [Python und C/C++](node13.ipynb) **Previous:** [ctypes](node14.ipynb)

##  SWIG
Um eine C- oder C++-Biliothek an Python anzubinden, verwenden wir den SWIG-Interface-Generator (SWIG = Simplified Wrapper and Interface Generator). Dieses Modul ist nicht Bestandteil der Python-Standard-Installation und muss getrennt installiert werden, z.B. mit   ``sudo apt-get install swig``  auf Ubuntu-Systemen. (Es ist nicht auf Python beschränkt, sondern kann auch als Interface zu anderen Sprachen genutzt werden.)
 
### C-Klassen
Wir verwenden das Beispiel aus dem [Tutorial](http://www.swig.org/tutorial.html).
Die Datei  ``example.c``  soll in Python eingebunden werden:

```C++
/* File : example.c */
 
 #include <time.h>
 double My_variable = 3.0;
 
 int fact(int n) {
     if (n <= 1) return 1;
     else return n*fact(n-1);
 }
 
 int my_mod(int x, int y) {
     return (x%y);
 }
        
 char *get_time()
 {
     time_t ltime;
     time(&ltime);
     return ctime(&ltime);
 }
```

 Um die Funktionen  ``fact, my_mod, get_time``  und die Variable   ``My_variable``  aus  ``example.c``  zu wrappen, benötigen wir folgende Interface-Datei  ``example.i`` :

```C++
/* example.i */
 %module example
 %{
 /* Put header files here or function declarations like below */
 extern double My_variable;
 extern int fact(int n);
 extern int my_mod(int x, int y);
 extern char *get_time();
 %}
 
 extern double My_variable;
 extern int fact(int n);
 extern int my_mod(int x, int y);
 extern char *get_time();
```

Die `%module`-Direktive definiert den Namen des zu generierenden Moduls und enthält weiteren Code (Header und Deklarationen), der in den von SWIG erzeugten Wrapper kopiert wird. 
Der [zweite Teil](https://stackoverflow.com/questions/32208350/swig-why-we-need-to-declare-functions-twice) enthält die Definitionen derjenigen Funktionen, für die ein Wrapper erzeugt werden soll.

SWIG kann mit dieser Interface-Datei ,glue-code' für einige Sprachen erzeugen (z.B. Perl, Java). Für Python wird SWIG folgendermaßen  aufgerufen und erzeugt dabei die Dateien  ``example.py``  und   ``example_wrap.c.`` :
```bash
swig -python example.i
```
 Nun muss  ``example_wrap.c.``  zu einer ,shared library' mit dem Namen  ``_example.so``  kompiliert werden:
```bash
gcc -fPIC -c example.c example_wrap.c -I/usr/include/python2.7/
ld -shared example.o example_wrap.o -o _example.so
```

 Innerhalb von Python ist das neue Modul  ``example``  und die entsprechenden Routinen nun folgendermaßen aufrufbar:

In [ ]:
# this example only works with Python2 kernel (on my machine / with my version of swig)
import sys
print(sys.version_info)
assert(sys.version_info.major == 2)

In [ ]:
from source import example
example.fact(5)

In [ ]:
dir(example)

In [ ]:
example.my_mod(7,3)

In [ ]:
example.get_time()

In [ ]:
example.cvar.My_variable

Alles, was im  ``%{ ... % }`` -Block steht, wird unverändert in die  automatisch generierte  ``example_wrap.c``  eingefügt.   
 
### Automatisch kompilieren
Um den gesamten Programmcode auf einmal zu kompilieren und zu einer shared library zusammenzufassen, kann folgendes Python-Programm   ``example_setup.py``  (verwendet  ``distutils`` ) benutzt werden:
```python
#!/usr/bin/env python
from distutils.core import setup, Extension
example_module = Extension('_example', sources = ['example.c','example_wrap.c'])
setup ( name = 'example',
        version = '0.1',
        author = 'John Doe',
        description = "An example program",
        ext_modules = [example_module],
        py_modules = ['example']
)
```

Mit folgender Zeile erzeugt man dann das module  ``example`` :
```bash
python example_setup.py build_ext --inplace
```

### C++-Klassen

C++-Klassen werden genauso wie C-Klassen eingebunden. Eine Klasse   ``Person``  wird in Header-Datei [person.h](source/person.h) deklariert. Die Member-Funktionen werden in der Datei  [person.cxx](source/person.cxx) deklariert. Ein  Testprogramm [person_test.cxx](source/person_test.cxx) demonstiert die Benutzung der Klasse  ``Person`` .   Diese C++-Programm lässt sich folgendermassen übersetzen und starten:
```bash
g++ person.cxx person_test.cxx -o person_test
./person_test
```

 Im SWIG-Wrapper [person.i](source/person.i) wird die ganze Klasse  ``Person``  unverändert definiert. Um den Kode automatisch zu kompilieren, wird die Datei   [person_setup.py](source/person_setup.py) verwendet.   Der SWIG glue code wird mit folgender Zeile erzeugt:

```bash
swig -python -c++ person.i
```

Das gesamte Modul  ``person``  wird mit folgendem Befehl erzeugt:
```bash
python person_setup.py build_ext --inplace
```

 Innerhalb von Python kann das Modul  ``Person``  folgendermaßen verwendet  werden:

In [ ]:
from source.person import Person
p=Person('Max Student')
p.get_name()

In [ ]:
p.set_name('Maria Studentin')
p.get_name()

In [ ]:
p2=Person(p)
p2.get_name()

In [ ]:
# quit() causes to the kernel to stop.
# We use it to see that the destructor is called.
# (Check the output in the console where you started the notebook server.)
quit()